In [20]:
origin_df = pd.read_csv('/home/hwang/leem/level2-objectdetection-cv-18/2-stages/level1-imageclassification-cv-18/post-processing/WBF_ATSS.csv')
print(origin_df.head())

                                    PredictionString       image_id
0  7 0.8395863175392151 215.65679931640625 50.585...  test/0000.jpg
1  4 0.6536312103271484 344.48748779296875 249.65...  test/0001.jpg
2  1 0.7134121656417847 424.9185485839844 269.755...  test/0002.jpg
3  9 0.5041786432266235 139.1294708251953 264.084...  test/0003.jpg
4  0 0.5496219396591187 426.969482421875 407.7453...  test/0004.jpg


In [10]:
import pandas as pd
import os
from PIL import Image

# CSV 파일 로드
df = pd.read_csv('/home/hwang/leem/level2-objectdetection-cv-18/2-stages/level1-imageclassification-cv-18/post-processing/WBF_ATSS.csv')

# PredictionString을 6개 단위로 나누는 함수 정의
def split_prediction_string(prediction_string):
    # 공백으로 나누어 리스트로 변환
    split_values = prediction_string.split()
    
    # 6개 단위로 묶어서 반환 (class_id, confidence_score, x1, y1, width, height)
    return [split_values[i:i+6] for i in range(0, len(split_values), 6)]

root_dir = "/hdd1/lim_data/level2_dataset/test"
# 각 PredictionString에 대해 함수 적용하여 bbox_info 열 추가
# df['image_id'] = df['image_id'].apply(lambda x: os.path.join(root_dir, x))
df['bbox_info'] = df['PredictionString'].apply(split_prediction_string)

# 결과 출력
print(df[['image_id', 'bbox_info']].head())

        image_id                                          bbox_info
0  test/0000.jpg  [[7, 0.8395863175392151, 215.65679931640625, 5...
1  test/0001.jpg  [[4, 0.6536312103271484, 344.48748779296875, 2...
2  test/0002.jpg  [[1, 0.7134121656417847, 424.9185485839844, 26...
3  test/0003.jpg  [[9, 0.5041786432266235, 139.1294708251953, 26...
4  test/0004.jpg  [[0, 0.5496219396591187, 426.969482421875, 407...


In [2]:
class_to_new_id = {
    'General trash': 0,
    'Paper': 1,
    'Paper pack': 2,
    'Metal': 3,
    'Glass': 4,
    'Plastic': 5,
    'Styrofoam': 6,
    'Plastic bag': 7,
    'Battery': 8,
    'Clothing': 9
}



In [3]:
output_dir = '/hdd1/lim_data/level2_dataset/crop_ATSS_test'
os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리가 없으면 생성

cropped_data = []
def crop_and_save_images(image_id, bbox_info_list):
    # 이미지 파일 경로 설정
    image_path = os.path.join("/hdd1/lim_data/level2_dataset", image_id)  # 'your_image_folder'에 실제 이미지가 위치
    img = Image.open(image_path)  # 이미지 로드

    # 각 bounding box에 대해 크롭 후 저장
    for idx, bbox_info in enumerate(bbox_info_list):
        class_id, confidence, x1, y1, width, height = map(float, bbox_info)
        x1, y1, width, height = int(x1), int(y1), int(width), int(height)

        # Bounding box로 이미지 크롭
        cropped_img = img.crop((x1, y1, x1 + width, y1 + height))

        # 크롭된 이미지 저장 (파일 이름에 image_id와 bbox 번호 포함)
        cropped_img_name = f"{os.path.splitext(os.path.basename(image_id))[0]}_crop_{idx}.jpg"
        cropped_img_path = os.path.join(output_dir, cropped_img_name)
        cropped_img.save(cropped_img_path)

        cropped_data.append({
            'label': int(class_id),  # label에 class_id를 임시로 저장
            'new_image_path': f"crop_test/{cropped_img_name}",
            'label_id': int(class_id),
            'cs' : confidence
        })

In [18]:
# df.apply(lambda row: crop_and_save_images(row['image_id'], row['bbox_info']), axis=1)

In [4]:
from tqdm import tqdm 

for index, row in tqdm(df.iterrows(), total=len(df)):
    crop_and_save_images(row['image_id'], row['bbox_info'])

100%|██████████| 4871/4871 [26:51<00:00,  3.02it/s] 


In [5]:
# DataFrame으로 변환 후, class_id를 label로 변환
cropped_df = pd.DataFrame(cropped_data)

# class_id를 label로 변환
cropped_df['label'] = cropped_df['label'].replace({v: k for k, v in class_to_new_id.items()})

In [6]:
csv_name = "/hdd1/lim_data/level2_dataset/csv/WBF_ATSS_test.csv"
cropped_df.to_csv(csv_name, index=False)
print(cropped_df.head())

         label             new_image_path  label_id        cs
0  Plastic bag  crop_test/0000_crop_0.jpg         7  0.839586
1  Plastic bag  crop_test/0000_crop_1.jpg         7  0.793338
2        Paper  crop_test/0000_crop_2.jpg         1  0.773484
3  Plastic bag  crop_test/0000_crop_3.jpg         7  0.771402
4  Plastic bag  crop_test/0000_crop_4.jpg         7  0.762092


In [15]:
import os
import pandas as pd
from tqdm import tqdm

def save_bbox_info_to_csv(df, output_csv_path):
    # CSV에 저장할 데이터를 담을 리스트
    cropped_data = []
    
    # 각 row에 대해 bbox 정보를 저장
    for index, row in tqdm(df.iterrows(), total=len(df)):
        image_id = row['image_id']
        bbox_info_list = row['bbox_info']

        for idx, bbox_info in enumerate(bbox_info_list):
            class_id, confidence, x1, y1, width, height = map(float, bbox_info)

            cropped_data.append({
                'image_id': os.path.basename(image_id),  # image_id의 base 경로만 저장
                'idx' : idx,
                'class_id': int(class_id),
                'confidence': confidence,
                'x1': x1,
                'y1': y1,
                'width': width,
                'height': height
            })

    # DataFrame으로 변환 후 CSV로 저장
    cropped_df = pd.DataFrame(cropped_data)
    cropped_df.to_csv(output_csv_path, index=False)

    return cropped_df

In [17]:

output_csv_path = "/home/hwang/leem/level2-objectdetection-cv-18/2-stages/classification/WBF_ATSS_bbox_info.csv"
convert_df = save_bbox_info_to_csv(df, output_csv_path)
print(convert_df.head())

100%|██████████| 4871/4871 [00:00<00:00, 7035.24it/s]


   image_id  idx  class_id  confidence          x1          y1       width  \
0  0000.jpg    0         7    0.839586  215.656799   50.585876  455.092651   
1  0000.jpg    1         7    0.793338  448.197571  602.263062  647.875061   
2  0000.jpg    2         1    0.773484  554.609985  104.805801  755.877747   
3  0000.jpg    3         7    0.771402  604.898438  517.767456  957.673035   
4  0000.jpg    4         7    0.762092  390.148102  191.706436  611.041016   

        height  
0   471.122375  
1   875.052856  
2   360.105255  
3  1022.789124  
4   546.604614  


In [22]:
def convert_to_prediction_string_format(cropped_df, output_csv_path):
    # 새로 변환할 데이터를 담을 리스트
    prediction_data = []
    
    # image_id를 기준으로 그룹화하여 PredictionString을 생성
    grouped = cropped_df.groupby('image_id')
    
    for image_id, group in grouped:
        # 각 이미지에 대한 bbox 정보들을 PredictionString으로 결합
        prediction_string = " ".join(
            group.apply(lambda row: f"{int(row['class_id'])} {row['confidence']} {int(row['x1'])} {int(row['y1'])} {int(row['width'])} {int(row['height'])}", axis=1)
        )
        
        prediction_data.append({
            'PredictionString': prediction_string,
            'image_id': image_id
            
        })
    
    # DataFrame으로 변환 후 CSV로 저장
    prediction_df = pd.DataFrame(prediction_data)
    prediction_df.to_csv(output_csv_path, index=False)

    return prediction_df

In [25]:
# 앞서 저장한 bbox 정보 CSV 파일을 불러오거나 만든 DataFrame을 사용
cropped_df = pd.read_csv("/home/hwang/leem/level2-objectdetection-cv-18/2-stages/classification/WBF_ATSS_bbox_info.csv")
output_csv_path = "/home/hwang/leem/level2-objectdetection-cv-18/2-stages/classification/converted_WBF_ATSS.csv"

# 분리된 데이터를 다시 PredictionString 형식으로 변환
prediction_df = convert_to_prediction_string_format(cropped_df, output_csv_path)

# 변환된 데이터 확인
print(prediction_df.head())

                                    PredictionString  image_id
0  7 0.8395863175392151 215 50 455 471 7 0.793337...  0000.jpg
1  4 0.6536312103271484 344 249 753 695 5 0.56312...  0001.jpg
2  1 0.7134121656417847 424 269 641 325 0 0.66376...  0002.jpg
3  9 0.5041786432266235 139 264 893 820 9 0.25488...  0003.jpg
4  0 0.5496219396591187 426 407 656 574 1 0.35824...  0004.jpg


In [26]:
import pandas as pd

def compare_csv_format(file1_path, file2_path):
    # 두 CSV 파일 로드
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)
    
    # 1. 컬럼 이름과 순서 비교
    columns_file1 = df1.columns.tolist()
    columns_file2 = df2.columns.tolist()
    
    if columns_file1 != columns_file2:
        print("❌ 두 CSV 파일의 컬럼 이름과 순서가 다릅니다.")
        print(f"파일 1의 컬럼: {columns_file1}")
        print(f"파일 2의 컬럼: {columns_file2}")
        return
    else:
        print("✅ 두 CSV 파일의 컬럼 이름과 순서가 동일합니다.")
    
    # 2. 행(row)과 열(column) 개수 비교
    if df1.shape != df2.shape:
        print(f"❌ 두 CSV 파일의 크기가 다릅니다. (파일 1: {df1.shape}, 파일 2: {df2.shape})")
        return
    else:
        print(f"✅ 두 CSV 파일의 크기(행과 열 개수)가 동일합니다. (크기: {df1.shape})")
    
    # 3. 첫 번째 열의 문자열 형식 비교 (공백 개수 확인)
    first_column_file1 = df1.iloc[:, 0].astype(str)
    first_column_file2 = df2.iloc[:, 0].astype(str)
    
    mismatch_count = 0
    
    for i in range(len(first_column_file1)):
        space_count_file1 = first_column_file1[i].count(' ')
        space_count_file2 = first_column_file2[i].count(' ')
        
        if space_count_file1 != space_count_file2:
            print(f"❌ 파일 1과 파일 2의 첫 번째 열에서 공백 개수가 다릅니다. (행 {i + 1})")
            print(f"파일 1: '{first_column_file1[i]}' (공백 {space_count_file1}개)")
            print(f"파일 2: '{first_column_file2[i]}' (공백 {space_count_file2}개)")
            mismatch_count += 1
            
    if mismatch_count == 0:
        print("✅ 첫 번째 열의 문자열 형식(공백 개수)이 두 파일에서 동일합니다.")
    else:
        print(f"총 {mismatch_count}개의 행에서 공백 개수가 다릅니다.")
    
# 사용 예시
file1_path = "/home/hwang/leem/level2-objectdetection-cv-18/2-stages/classification/converted_WBF_ATSS.csv"
file2_path = "/home/hwang/leem/level2-objectdetection-cv-18/2-stages/level1-imageclassification-cv-18/post-processing/WBF_ATSS.csv"
compare_csv_format(file1_path, file2_path)

✅ 두 CSV 파일의 컬럼 이름과 순서가 동일합니다.
✅ 두 CSV 파일의 크기(행과 열 개수)가 동일합니다. (크기: (4871, 2))
✅ 첫 번째 열의 문자열 형식(공백 개수)이 두 파일에서 동일합니다.
